[A Review of Singular Value Decomposition](#A-Review-of-Singular-Value-Decomposition)

1. [SVD](#SVD)  
2. [Performing SVD Using numpy](#Performing-SVD-Using-numpy)     
3. [SVD's Relationship to $A^TA$](#SVD's-Relationship-to-$A^TA$)   
4. [Dimensionality Reduction](#Dimensionality-Reduction)   
5. [Dimensionality Reduction - Reducing the Number of Columns](#Dimensionality-Reduction---Reducing-the-Number-Columns)   



# A Review of Singular Value Decomposition

Before moving on to Latent Semantic Analysis I want to take a moment to review the workhorse behind that algorithm, Singular Value Decompositions (SVD). The SVD is a way to decompose a matrix that is incredibly valuable in data science and machine learning. Even if you've haven't heard of the SVD before, you've almost certainly used it. Anytime you use `sklearn`'s `PCA` you're running your data matrix through an SVD decomposition! Let's get going and dive into SVD.

## SVD
Suppose you have an $m\times n$ real matrix, $A$. Then it is a fact that:

$$
A = U\Sigma^TV^T
$$

where $U$ is an $m \times m$ orthogonal real matrix ($UU^T = I$), $V$ is an $n \times n$ orthogonal real matrix and $\Sigma$ is an $m\times n$ diagonal matrix of the same rank as $A$. The entries of $\Sigma$ are known as the singular values of $A$ and are denoted as $\sigma_i$ for $i = 1, \dots,p$, $p= \min{m,n}$. It is standard to let $\sigma_1\geq \sigma_2\geq \cdots \geq \sigma_p$. The singular values of $A$ are the square roots of the nonzero eigenvalues of $A^TA$ and $AA^T$.

The columns of $U$ are known as the left singular vectors of $A$ and the columns of $V$ are known as the right singular vectors of $A$.

## Performing SVD Using numpy



In [1]:
# import the package
import numpy as np

np.random.seed(440)

In [2]:
# Make a random matrix
A = np.random.randn(100,4)

In [3]:
# np.linalg.svd returns the svd decomposition of a matrix
# Note that linalg.svd returns V transpose not V itself
U,S,Vt = np.linalg.svd(A)

In [4]:
V = Vt.transpose()

In [5]:
np.shape(U)

(100, 100)

In [6]:
U

array([[ 0.05002525, -0.21853099,  0.06034972, ..., -0.02804724,
         0.09004994, -0.06492556],
       [-0.06531026,  0.14653637,  0.00867559, ..., -0.02274976,
         0.01327923,  0.20871252],
       [-0.0927834 ,  0.02544625,  0.1902059 , ..., -0.14423587,
        -0.00892435,  0.07629933],
       ...,
       [-0.02438868,  0.04858554,  0.16979483, ...,  0.97124306,
         0.02040163,  0.01618576],
       [-0.11071266,  0.06340291, -0.14022391, ...,  0.01438983,
         0.96715306, -0.00888423],
       [-0.0606144 , -0.18896709, -0.07498754, ...,  0.02069631,
        -0.00612093,  0.94749458]])

In [7]:
np.shape(S)

(4,)

In [8]:
S

array([11.89845699,  9.91852466,  9.40770237,  7.88849121])

In [9]:
np.shape(V)

(4, 4)

## SVD's Relationship to $A^TA$
SVD has a nice relationship with $A^TA$.

If the SVD of $A$ is $U\Sigma^TV^T$ then:

$$
A^TA = (V\Sigma^TU^T)(U\Sigma V^T)= V\Sigma^2V^T
$$


because $U$ is an orthogonal matrix, and thus $\Sigma^2$ holds the eigenvalues of $A^TA$ and $V$ holds the corresponding eigenvectors.

Similarly:

$$
AA^T = U\Sigma^2U^T
$$

and thus $\Sigma^2$ also holds the eigenvalues of $AA^T$and $U$ holds the corresponding eigenvectors.

### Note on PCA
If $A$ is a data matrix with centered columns then the above fact shows SVD's relationship to principal components analysis (PCA).

## Dimensionality Reduction
SVD is incredibly useful for dimensionality reduction. When I was preparing for this notebook I easily got confused because there are a couple different things people mean by dimensionality reduction when it comes to SVD. Let's first touch on the data compression interpretation.

### Data Compression
Consider the following theorem due to Eckart and Young.

Suppose the SVD of $A$ is as described above. If $k<r = rank(A)$ and $A_k= \sum_{i=1}^{k}\sigma_iu_iv_i^T$, then

$$
\min_{rank(B)=k}||A-B||_2 = |||A-A_k||_2 = \sigma{k+1}
$$


_Fun fact: You can use this to compute the explained variance ratio for PCA_.

In [10]:
# Let's "test" this out with our matrix decomposition!
test = S[0]*U[:,0].reshape(-1,1).dot(V[:,0].reshape(-1,1).transpose())

print(np.linalg.norm(test - A, 2))
print(S[1])

9.918524656488632
9.918524656488636


In [11]:
print(S[2])

9.407702367006781


Now this is where I got confused. When I think of dimensionality reduction, I don't picture this. If $A$ is a data matrix, $A_k$ has the same dimensionality as $A$  (same number of columns). However, it is data reduction in the sense that I can compress the amount of data I need to store for a decent approximation of $A$ by using a lower rank matrix. This approach is used in a lot of fields. One example is image compression http://www.math.utah.edu/~goller/F15_M2270/BradyMathews_SVDImage.pdf.

## Dimensionality Reduction - Reducing the Number of Columns
Let's see how we can perform dimensionality reduction in the way that we usually think about it in data science by considering the sample matrix we've been using in this notebook.

Now suppose we want to reduce our example $A$ from a $100 \times 4$  to a $100 \times 2$ matrix. We can do this by multiplying the first $2$ columns of $U$ with the upper left $2 \times 2$ block of $\Sigma$, i.e. we reduce the dimensions of $A$ by calculating $U_2\times V_2$.

In [12]:
Sigma = np.zeros(np.shape(A))
for i in range(len(S)):
    Sigma[i,i] = S[i]

In [13]:
U[:,:2].dot(Sigma[:2,:2])

array([[ 0.59522334, -2.167505  ],
       [-0.7770913 ,  1.45342457],
       [-1.10397927,  0.25238929],
       [-2.02259354, -0.09840806],
       [-0.24364657,  0.309503  ],
       [-0.22183936,  1.02617724],
       [-0.08698947, -0.10528557],
       [ 1.60409758, -0.49764038],
       [-0.9538455 ,  2.15436326],
       [ 0.11765662,  1.08212882],
       [ 0.23696583, -1.5598674 ],
       [ 1.4843431 ,  1.10806291],
       [-1.20535429, -1.83027665],
       [ 0.64046282,  0.926068  ],
       [-0.44976983,  0.50797085],
       [-1.39870551, -0.07690773],
       [ 0.663437  ,  1.70818103],
       [ 0.18866595,  1.75661591],
       [-1.28170834, -0.49639267],
       [ 1.10784759, -0.84294297],
       [-1.17973897,  0.35227067],
       [ 0.50294366,  0.4702022 ],
       [-0.10044903, -1.62529895],
       [-0.17302177,  0.1979349 ],
       [ 1.7060943 , -0.03834834],
       [-0.95074556, -0.30227541],
       [ 1.74460052,  1.12033494],
       [-0.4356569 , -1.78645691],
       [ 0.7891667 ,

You can do this in general by keeping the first $k$ columns of $U$ and the upper right $k\times k$ block of $\Sigma$, for $k \leq rank{A}$.

### Alternatively
Another way to reduce $A$ to a lower dimensional matrix using SVD is to use `sklearn`'s `TruncatedSVD` object. While we can do this by hand using `numpy`, `TruncatedSVD` is more efficient.

In [14]:
from sklearn.decomposition import TruncatedSVD

In [15]:
trunc_svd = TruncatedSVD()

In [16]:
trunc_svd.fit_transform(A)

array([[ 0.59522334,  2.167505  ],
       [-0.7770913 , -1.45342457],
       [-1.10397927, -0.25238929],
       [-2.02259354,  0.09840806],
       [-0.24364657, -0.309503  ],
       [-0.22183936, -1.02617724],
       [-0.08698947,  0.10528557],
       [ 1.60409758,  0.49764038],
       [-0.9538455 , -2.15436326],
       [ 0.11765662, -1.08212882],
       [ 0.23696583,  1.5598674 ],
       [ 1.4843431 , -1.10806291],
       [-1.20535429,  1.83027665],
       [ 0.64046282, -0.926068  ],
       [-0.44976983, -0.50797085],
       [-1.39870551,  0.07690773],
       [ 0.663437  , -1.70818103],
       [ 0.18866595, -1.75661591],
       [-1.28170834,  0.49639267],
       [ 1.10784759,  0.84294297],
       [-1.17973897, -0.35227067],
       [ 0.50294366, -0.4702022 ],
       [-0.10044903,  1.62529895],
       [-0.17302177, -0.1979349 ],
       [ 1.7060943 ,  0.03834834],
       [-0.95074556,  0.30227541],
       [ 1.74460052, -1.12033494],
       [-0.4356569 ,  1.78645691],
       [ 0.7891667 ,

### Sign, Sign Everywhere a Sign
You may notice that the signs of the two computations are opposite. That happens sometimes as the sign is determined by a random process in the code behind the scenes. This is fine in principle since the vectors will be the same but flipped.

### But Why?
So why are we doing this? That's a good question. I've tried to search around and figure out why we take this approach for dimensionality reduction with SVD since we're seemingly leaving the  matrix out to dry. This is the best explanation I can come up with.

It holds that:

$$
AV = U\Sigma.
$$

For the reduction of  to  dimensions:

$$
AV_k = U_k\Sigma_k.
$$

The left hand side results in a matrix of the scalar projections of each observation from $A$ onto each column of $V_k$ (i.e. going from $n$ columns for each observation to $k$ columns for each observation). In the case of a scaled and cenetered data matrix, $A$, this is precisely the principal component values.

In [17]:
# We can check this like so
svd_test = trunc_svd.fit_transform(A - np.mean(A,axis=0)).round(4)

In [18]:
from sklearn.decomposition import PCA

In [19]:
pca = PCA()

pca_test = pca.fit_transform(A)[:,:2].round(4)

In [20]:
pca_test == svd_test

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ Tr

As a continuation of why we're looking at $U\Sigma$ for projecting into a lower dimensional space, it has to do with the typical organization of a data matrix. Typically, the rows of a data matrix correspond to individual observations, while the columns represent independent variables.

The right singular vectors (columns of $V$) represent the orthogonal directions in the original dataspace that best represent the original data in that lower dimension, i.e. are closest to $A$ in $2$-norm.

So when we project onto the first $k$ columns of $V$ (compute $U_k\Sigma_k$, we're projecting into the $k$-dimensional space that most closely approximates $A$ in $2$-norm.

### Ahh, How Refreshing
Now that we've refreshed ourselves about SVD let's return to semantic analysis. If you'd like to read more about SVD check out the References Folder.